In [1]:
# Import dependencies
from sklearn import tree
import pandas as pd
import numpy as np
import os
import warnings
import timeit
from IPython.display import display 
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5000)

In [2]:
# Read in the data as a dataframe.
df = pd.read_csv('NRD_2017_Mental_Health_Burden_with_admission_reasons.csv')
df.head()

,Unnamed: 0,key_nrd,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd_x,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum_x,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year_x,zipinc_qrtl,dxver,prver,hosp_bedsize,h_contrl,hosp_urcat4,hosp_ur_teach,nrd_stratum_y,n_disc_u,n_hosp_u,s_disc_u,s_hosp_u,total_disc,year_y,aprdrg,aprdrg_risk_mortality,aprdrg_severity,hosp_nrd_y,next_nrdday,days_to_readmission,readmission_diagnosis,readmission_los,preventable_admission,pqi_1,pqi_3,pqi_5,pqi_7,pqi_8,pqi_11,pqi_12,pqi_14,pqi_15,pqi_16,pqi_5_15_exclusion,pqi_7_medical_exclusions,pqi_7_procedural_exclusion,pqi_7_8_procedural_exclusion,pqi_11_exclusion,pqi_12_exclusion,pqi_immunocompromise_medical_exc,pqi_immunocompromise_procedural_,pqi_16_exclusion_procedure,pqi_16_amputation_procedure,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,cc_mi,cc_chf,cc_pvd,cc_cvd,cc_dementia,cc_copd,cc_rheum,cc_peptic_ulcer,cc_mild_liver_disease,cc_diabetes_wo_complications,cc_diabetes_w_complications,cc_plegia,cc_renal,cc_malignancy,cc_severe_liver,cc_metastasis,cc_aids,ibd_diagnosis,ibd_index_hospitalization,intestinal_abscess,meningitis,encephalitis,endocarditis,pneumonia,pyelonephritis,septic_arthritis_osteomyelitis,septicemia_or_bacteremia,clostridium_difficile,pulmonary_tuberculosis,atypical_mycobacteria,cryptococcosis_aspergillosis_his,listeriosis,leishmaniasis,pneumocystis_jiroveci_pneumonia,acute_myocardial_infarction,heart_failure,unstable_angina,arrhythmia,cardiac_arrest,vte_diagnosis_pe_or_dvt,history_of_vte,systemic_steroid_use,chronic_pain,opioid_overuse,depression,anxiety_disorder,bipolar_disorder,schizophrenia,problems_related_to_education_an,problems_related_to_employment_a,occupational_exposure_to_risk_fa,problems_related_to_housing_and_,problems_related_to_social_envir,problems_related_to_upbringing,other_problems_related_to_primar,problems_related_to_certain_psyc,problems_related_to_other_psycho,intestinal_surgery,flexible_sigmoidoscopy_colonosco,blood_transfusion,parenteral_or_enteral_nutrition,serious_infections_during_follow,total_serious_infection_admissio,opportunistic_infections_during_,total_opportunistic_infection_ad,cardiac_during_followup,total_cardiac_admissions,venous_thromboembolism_during_fo,total_venous_thromboembolism_adm,steroid_use_during_followup,total_steroid_use_admissions,chronic_pain_during_followup,total_chronic_pain_admissions,opioid_overuse_during_followup,total_opioid_overuse_admissions,psychiatric_burden_during_follow,total_psychiatric_burden_admissi,social_determinants_of_health_du,total_social_determinants_of_hea,intestinal_surgery_during_follow,total_intestinal_surgery_admissi,v250,total_flexible_sigmoidoscopy_col,blood_transfusion_during_followu,total_blood_transfusion_admissio,parenteral_or_enteral_nutrition_,total_parenteral_or_enteral_nutr,severe_ibd_hospitalization,tot_admits_for_pt,tot_hosp_charges_for_pt,tot_los_pt,cd_patients_only,uc_patients_only,readm_thirty_days,readm_ninety_days,total_30_day_readm,total_90_day_readm,tot_proc_for_pt,total_surg_for_pt,mental_health_opiate_for_pt,readm_during_fu,days_to_first_readm_during_fu,alive_fu_time,tot_los_per_ali

In [3]:
# Drop unnecessary columns.
df = df.drop(columns=('Unnamed: 0'))
df.head()

,key_nrd,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd_x,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum_x,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year_x,zipinc_qrtl,dxver,prver,hosp_bedsize,h_contrl,hosp_urcat4,hosp_ur_teach,nrd_stratum_y,n_disc_u,n_hosp_u,s_disc_u,s_hosp_u,total_disc,year_y,aprdrg,aprdrg_risk_mortality,aprdrg_severity,hosp_nrd_y,next_nrdday,days_to_readmission,readmission_diagnosis,readmission_los,preventable_admission,pqi_1,pqi_3,pqi_5,pqi_7,pqi_8,pqi_11,pqi_12,pqi_14,pqi_15,pqi_16,pqi_5_15_exclusion,pqi_7_medical_exclusions,pqi_7_procedural_exclusion,pqi_7_8_procedural_exclusion,pqi_11_exclusion,pqi_12_exclusion,pqi_immunocompromise_medical_exc,pqi_immunocompromise_procedural_,pqi_16_exclusion_procedure,pqi_16_amputation_procedure,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,cc_mi,cc_chf,cc_pvd,cc_cvd,cc_dementia,cc_copd,cc_rheum,cc_peptic_ulcer,cc_mild_liver_disease,cc_diabetes_wo_complications,cc_diabetes_w_complications,cc_plegia,cc_renal,cc_malignancy,cc_severe_liver,cc_metastasis,cc_aids,ibd_diagnosis,ibd_index_hospitalization,intestinal_abscess,meningitis,encephalitis,endocarditis,pneumonia,pyelonephritis,septic_arthritis_osteomyelitis,septicemia_or_bacteremia,clostridium_difficile,pulmonary_tuberculosis,atypical_mycobacteria,cryptococcosis_aspergillosis_his,listeriosis,leishmaniasis,pneumocystis_jiroveci_pneumonia,acute_myocardial_infarction,heart_failure,unstable_angina,arrhythmia,cardiac_arrest,vte_diagnosis_pe_or_dvt,history_of_vte,systemic_steroid_use,chronic_pain,opioid_overuse,depression,anxiety_disorder,bipolar_disorder,schizophrenia,problems_related_to_education_an,problems_related_to_employment_a,occupational_exposure_to_risk_fa,problems_related_to_housing_and_,problems_related_to_social_envir,problems_related_to_upbringing,other_problems_related_to_primar,problems_related_to_certain_psyc,problems_related_to_other_psycho,intestinal_surgery,flexible_sigmoidoscopy_colonosco,blood_transfusion,parenteral_or_enteral_nutrition,serious_infections_during_follow,total_serious_infection_admissio,opportunistic_infections_during_,total_opportunistic_infection_ad,cardiac_during_followup,total_cardiac_admissions,venous_thromboembolism_during_fo,total_venous_thromboembolism_adm,steroid_use_during_followup,total_steroid_use_admissions,chronic_pain_during_followup,total_chronic_pain_admissions,opioid_overuse_during_followup,total_opioid_overuse_admissions,psychiatric_burden_during_follow,total_psychiatric_burden_admissi,social_determinants_of_health_du,total_social_determinants_of_hea,intestinal_surgery_during_follow,total_intestinal_surgery_admissi,v250,total_flexible_sigmoidoscopy_col,blood_transfusion_during_followu,total_blood_transfusion_admissio,parenteral_or_enteral_nutrition_,total_parenteral_or_enteral_nutr,severe_ibd_hospitalization,tot_admits_for_pt,tot_hosp_charges_for_pt,tot_los_pt,cd_patients_only,uc_patients_only,readm_thirty_days,readm_ninety_days,total_30_day_readm,total_90_day_readm,tot_proc_for_pt,total_surg_for_pt,mental_health_opiate_for_pt,readm_during_fu,days_to_first_readm_during_fu,alive_fu_time,tot_los_per_alive_fu_pt,de

In [4]:
# Create a list of columns from the dataframe.
column_list = df.columns
column_list

Index(['key_nrd', 'age', 'aweekend', 'died', 'discwt', 'dispuniform', 'dmonth',
       'dqtr', 'drg', 'drgver',
       ...
       'adm_reason_endo_metabolism', 'adm_reason_fracture',
       'adm_reason_gastrointestinal', 'adm_reason_genitourinary',
       'adm_reason_ibd', 'adm_reason_infection', 'adm_reason_malignancies',
       'adm_reason_neuropsychiatric', 'adm_reason_respiratory',
       'adm_reason_thromboembolism'],
      dtype='object', length=335)

In [5]:
# Define a function that will take in the following:
# - df = the dataframe the columns are located in.
# - list_of_columns = a list of column names that will be used to find diagnosis/procedure codes from the dataframe.
# - column_label = a string value that will be used to initialize and create a new column that stores either a 1 or 0 depending on if the patient has a specific diagnosis/procedure.
# - search_codes = a single string value with the starting values for ICD-10 codes for the diagnosis/procedure that is being located.
#   - The format for search_codes should be as follows: 'code|code|code|code'; using the | separator allows individual searches and 'code' can be a partial code such as F80 instead
#     of every possible outcome like F801, F8009, F80901, etc.

# This function will create a new column with the desired name and populate it with a 1 or a 0 depending on if a patient has a specific diagnosis/procedure on that discharge.
def icd10_to_column(df, list_of_columns, column_label, search_codes):
    df[column_label] = 0
    for column in list_of_columns:
        # Since masking does not work with type NaN, convert column to string type.
        df[column] = df[column].astype('string')
        df.loc[df[column].str.contains(search_codes), column_label] = 1
    return print(f'Column for {column_label} created.')

In [6]:
# Create a list of diagnosis columns, all columns that begin with 'i10_dx'.
diagnoses = [column for column in column_list if 'i10_dx' in column]

# Create a list for the 'drg' column.
drg = ['drg']

In [7]:
# -- ADDITIONAL COLUMNS FOR VALIDATION TESTING BETWEEN 2013 AND 2017 DATA SETS -- #

# Thromboses
icd10_to_column(df, diagnoses, 'thromboses', 'I824|I260|I269|Z8671')

# Diabetes with chronic complications (cm_dmcx)
icd10_to_column(df, drg, 'cm_dmcx', '637|638')

# Drug abuse as a comorbidity (cm_drug)
icd10_to_column(df, drg, 'cm_drug', '895|896|897')

Column for thromboses created.
Column for cm_dmcx created.
Column for cm_drug created.
